Importing the libraries

In [1]:
pip install tensorflow datascience transformers numpy albumentations pandas nltk tensorflow_datasets

In [2]:
import tensorflow as tf

In [3]:
num_gpu_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num of GPU available :", num_gpu_available)
assert num_gpu_available > 0 

Num of GPU available : 1


In [4]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [5]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

In [6]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Getting the dataset

In [7]:
import tensorflow_datasets as tfds
ds = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', split='train', shuffle_files=True)
print(ds)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incomplete8RCVBQ/amazon_us_reviews-train.tfrecord


  0%|          | 0/104975 [00:00<?, ? examples/s]

Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.
<_OptionsDataset element_spec={'data': {'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'helpful_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'marketplace': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_category': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_parent': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_body': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_date': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_headline': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'star_rating': TensorSpec(shape=(), dtype=tf.int32, name=None), '

In [8]:
df = tfds.as_dataframe(ds)

In [9]:
df.head(4)

,data/customer_id,data/helpful_votes,data/marketplace,data/product_category,data/product_id,data/product_parent,data/product_title,data/review_body,data/review_date,data/review_headline,data/review_id,data/star_rating,data/total_votes,data/verified_purchase,data/vine
0,b'20980074',0,b'US',b'Mobile_Electronics',b'B00D1847NE',b'274617424',b'Teenage Mutant Ninja Turtles Boombox CD Play...,b'Does not work',b'2015-01-09',b'One Star',b'R1OVS0D6SEXPW7',1,0,0,1
1,b'779273',0,b'US',b'Mobile_Electronics',b'B00KMO6DYG',b'397452138',b'4 Gauge Amp Kit Amplifier Install Wiring Com...,b'This is a great wiring kit i used it to set ...,b'2015-08-06',b'Great kit',b'R9VSD0ET8FERB',4,0,0,1
2,b'15410531',0,b'US',b'Mobile_Electronics',b'B000GWLL0K',b'948304826',b'Travel Wall Charger fits Creative Zen Vision...,b'It works great so much faster than USB charg...,b'2007-03-15',b'A/C Charger for Creative Zen Vision M',b'R3ISXCZHWLJLBH',5,0,0,1
3,b'27389005',0,b'US',b'Mobile_Electronics',b'B008L3JE6Y',b'466340015',b'High Grade Robust 360\xc2\xb0 Adjustable Car...,b'This product was purchased to hold a monitor...,b'2013-07-30',b'camera stand',b'R1TWVUDOFJSQAW',5,0,0,1


Fetching relevant data

In [10]:
df['Sentiment'] = df["data/star_rating"].apply(lambda score: "positive" if score >=3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df['short_review'] = df['data/review_body'].str.decode("utf-8")
df = df[['short_review', "Sentiment"]]

In [11]:
df.head(4)

,short_review,Sentiment
0,Does not work,0
1,This is a great wiring kit i used it to set up...,1
2,It works great so much faster than USB charger...,1
3,This product was purchased to hold a monitor o...,1


In [12]:
n = 54975
df.drop(df.tail(n).index)

,short_review,Sentiment
0,Does not work,0
1,This is a great wiring kit i used it to set up...,1
2,It works great so much faster than USB charger...,1
3,This product was purchased to hold a monitor o...,1
4,it works but it has really bad sound quality. ...,1
...,...,...
49995,Awesome,1
49996,I got this cover for my daughter's ipod. It is...,1
49997,This battery does not fit the T-mobile G2 is i...,0
49998,"It has good sound but , the charge only last 1...",1


In [13]:
reviews = df['short_review'].values.tolist()
labels = df['Sentiment'].tolist()

Performing data split operation

In [14]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews,labels,test_size=0.2)

In [15]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Tokenizing

In [16]:
tokenizer([training_sentences[0]], truncation=True, padding=True, max_length=128 )

{'input_ids': [[101, 1045, 17402, 2023, 2005, 2026, 8148, 16351, 26322, 1012, 2349, 2000, 1996, 2946, 1997, 2023, 2553, 1010, 2009, 2001, 2081, 2005, 3080, 26322, 2015, 1012, 1996, 2126, 2027, 8081, 2023, 2003, 2011, 1037, 8903, 19274, 2008, 2017, 2404, 1999, 1996, 9474, 2553, 1998, 9909, 2946, 2029, 13956, 1996, 10947, 1010, 11754, 5017, 26322, 3572, 2039, 2153, 2015, 2023, 9474, 2553, 1012, 2009, 3475, 1005, 1056, 1996, 4602, 5576, 1010, 2021, 2009, 2515, 1996, 3105, 1998, 2573, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 2204, 1024, 2009, 2003, 1037, 2844, 1010, 25634, 1010, 8903, 2553, 2008, 2038, 1037, 3898, 3104, 2006, 2009, 1012, 2009, 2515, 1037, 14388, 3105, 8650, 1996, 26322, 1998, 1996, 3898, 1012, 1996, 8903, 2553, 2036, 3640, 2204, 15012, 2006, 9972, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Encoding

In [17]:
train_encodings = tokenizer(training_sentences, truncation=True, padding=True)
val_encodings = tokenizer(validation_sentences, truncation=True, padding=True)


In [18]:
print(train_encodings[0:10])

[Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=512, attrib

In [19]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), training_labels))#need to check x,y conert to numpy or vectorize

In [20]:
print(train_dataset)

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>


In [21]:
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), validation_labels))

Modelling

In [22]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-8)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

In [24]:
#model.fit(train_dataset.shuffle(100).batch(16), epochs=2, batch_size=16, validation_data=val_dataset.shuffle(100).batch(16))

In [25]:
model.save_pretrained("./sentiment")

In [26]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/sentiment")

Some layers from the model checkpoint at /content/sentiment were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/sentiment and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
test_sentence = "i like this item"

In [28]:
predict_input = tokenizer.encode(test_sentence, truncation=True, padding=True, return_tensors="tf")

In [29]:
tf_output = model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

Positive
